In [11]:
import networkx as nx
import numpy as np
import pandas as pd

import networkx.algorithms.community as nx_comm

import math

def entropy(nums):
    z = np.bincount(nums)
    N = len(nums)
    assert nums.shape == (N, )
    ent = 0.0
    for e in z:
        if e != 0:
            p = float(e) / N
            ent += p*math.log(p)
    assert ent <= 0
    ent = -ent

    assert ent >=0
    return ent
def computeNMI(clusters, classes):

    assert clusters.shape == classes.shape
    A = np.c_[(clusters, classes)]
    A = np.array(A)
    N = A.shape[0]
    assert A.shape == (N, 2)

    H_clusters = entropy(A[:, 0])
    H_classes = entropy(A[:, 1])
    # print H_clusters
    # print H_classes
    # assert N == 17
    NMI = 0.0
    for k in np.unique(A[:, 0]):
        # get elements in second column that have first column equal to j
        z = A[A[:, 0] == k, 1]
        len_wk = len(z)
        t = A[:, 1]
        #for each unique class in z
        for e in np.unique(z):

            wk_cj=len(z[z==e])
            len_cj=len(t[t == e])
            assert wk_cj <= len_cj
            numerator= (float(wk_cj) / float(N)) * math.log( (N*wk_cj) / float(len_wk * len_cj)  )
            NMI += numerator
    NMI /= float((H_clusters + H_classes) * 0.5)

    assert (NMI > 0.0 or abs(NMI) < 1e-10) and (NMI < 1.0 or abs(NMI - 1.0) < 1e-10)
    return NMI



def check_res(res,Adj):
    print(res)
    for i in range(0,len(res)):
        if Adj[i,res[i]] ==0:
            return False
    return True





def community_detection(nodes,edges,population=15,generation=30,r=1.5):
    
    graph=nx.Graph() 
    graph.add_nodes_from(nodes) #adds nodes
    graph.add_edges_from(edges) #add edges
    Adj = nx.adjacency_matrix(graph) 
   
    nodes_length = len(graph.nodes())
#     nx.draw(graph, with_labels=True,node_color = "red")
    d = {"chrom":[generate_chrom(nodes_length,Adj) for n in range(population)]}
      
    dframe = pd.DataFrame(data= d)
    dframe["subsets"] = dframe["chrom"].apply(find_subsets)
    
    
    dframe["community_score"]=dframe.apply(lambda x: community_score(x["chrom"],x["subsets"],r,Adj),axis=1)
#     completeData = pd.DataFrame(data=dframe)
    
    gen = 0
    population_count = population
#     k=population
    while gen < generation:
        for i in range(int(np.floor(population/10))):
            elites = dframe.sort_values("community_score",ascending=True)[int(np.floor(population/10)):]
            for index , parent in elites.iterrows():
                av_index = [i for i in elites.index]
                av_index.remove(index)
                target = av_index[np.random.randint(0,len(av_index))]
                av_index.remove(target)
                
                random1 = av_index[np.random.randint(0,len(av_index))]
                av_index.remove(random1)
                
                
                random2 = av_index[np.random.randint(0,len(av_index))]
                av_index.remove(random2)

                
#                 print(index , ' ' , target , ' ' , random1 , ' ' , random2)
              
                trial = mutation(dframe.loc[index],dframe.loc[target],dframe.loc[random1],dframe.loc[random2],Adj,0.5)
#                 print(trial)
                offspring = crossover(parent, trial, 0.8)
                off_subsets = find_subsets(offspring)
                off_c_score = community_score(offspring, off_subsets, r , Adj)
                dframe.loc[population_count]=[offspring, off_subsets,off_c_score]
                population_count += 1
                
#                 completeData.loc[k]=[offspring, off_subsets,off_c_score]
#                 k += 1
            
            
            
        dfsorted = dframe.sort_values("community_score",ascending=False)
        to_drop = dfsorted.index[population:]
        dframe.drop(to_drop,inplace=True)
        gen +=1   
#     print(dframe)
#     print(dfsorted)
#     print(dframe.sort_values("f_score",ascending=True))
    sorted_df = dframe.sort_values("community_score",ascending=False).index[0]
#     print('gnjgnioniogneiogeergnieginogrignigreio')
#     print(sorted_df)
    res = dframe.loc[sorted_df]
    
    
    nodes_subsets = res["subsets"]
    
    res = res['chrom']
    
    istrue = check_res(res,Adj)
    print(istrue)
    
    nodes_list = list(graph.nodes())
    result = []
    #print('nodeslist',nodes_list)
    for subs in nodes_subsets:
        subset = []
        for n in subs:
            subset.append(nodes_list[n])
        #print(subset)
        result.append(subset)
        
        
        
        
#     colors = ['red','blue','green','yellow','pink','orange','cyan','purple','grey','brown','olive','indigo','yellowgreen','chocolate']
#     alreadyvisited = []
#     k = 0 
#     colormap = ['']*result.size()
#     for i in result:
#         for j in i:
#             if j not in alreadyvisited:
#                 colormap[j] = colors[k]
#                 alreadyvisited.append(j)
#             else:
#                 colormap[j]=  'white'
#             j=j+1
#         k=k+1
#         i=i+1
#     nx.draw(graph,node_color = colormap , with_label = True)
#     plt.show()





    NMI = 0
    clu = dframe.loc[sorted_df]
    clu = clu['chrom']
    clu = np.array(clu)
    for index, target in dframe.iterrows():
        temp = np.array(target['chrom'])
        x = computeNMI(clu,temp)
        NMI += x
    NMI /= len(dframe)
    print('NMI')
    print(NMI)
    
    print('MODULARITY: ')
    
    modularity = nx_comm.modularity(graph, result)
    print(modularity)
    return result
































# returning the the array of column indexes where randomly chosen till value is 1 
def generate_chrom(nodes_length,Adj):
    chrom = np.array([],dtype=int)
    for x in range(nodes_length):
        rand = np.random.randint(0,nodes_length)
        while Adj[x,rand] != 1:
            rand = np.random.randint(0,nodes_length)
        chrom = np.append(chrom,rand)
    return chrom



def merge_subsets(sub):
    arr =[]
    to_skip=[]
    for s in range(len(sub)):
        if sub[s] not in to_skip:
            new = sub[s]
            for x in sub:
                if sub[s] & x:
                    new = new | x
                    to_skip.append(x)
            arr.append(new)
    return arr

def find_subsets(chrom):
    sub = [{x,chrom[x]} for x in range(len(chrom))]
    result=sub
    i=0
    while i<len(result):
        candidate = merge_subsets(result)
        if candidate != result:
            result = candidate
        else:
            break
        result=candidate
        i+=1
    return result

def community_score(chrom,subsets,r,Adj):
    matrix = Adj.toarray()
    CS=0
    for s in subsets:
        submatrix = np.zeros((len(chrom),len(chrom)),dtype=int)
        for i in s:
            for j in s:
                submatrix[i][j]=matrix[i][j]
        M=0
        v=0
        PS=0
        for row in list(s):
            ki = np.sum(matrix[row])
            kiin = np.sum(submatrix[row])
            kiout = ki - kiin
            P= kiin/ki
            PS+=P
            row_mean = kiin/len(s)
            v+=np.sum(submatrix[row])
            M+=(row_mean**r)/len(s)
        CS+=M*v
    OS= 0.5*CS/len(subsets) + 0.5*(1/PS)*len(subsets)  #Overall score is calculated by maximizing CS and min PS
    return OS

        
    
    

def roulette_selection(df_elites):
    prob = np.random.random_sample()
#     print('prob',prob)
    sum_cs=np.sum(df_elites["community_score"])
#     print(sum_cs)
    x=0
    selected = 0
    for i in df_elites.index:
        x += df_elites["community_score"][i]
            
        X=x/sum_cs
#         print('X',X)
        if prob < X:
            chosen=i
            break
    return chosen

def crossover(parent,trial,crossover_rate):
    if np.random.random_sample() >= crossover_rate:
        length = len(parent['chrom'])
        mask = np.random.randint(2, size=length)
        child = np.zeros(length,dtype=int)
        for i in range(len(mask)):
            if mask[i] == 1:
                child[i]=trial[i]
            else:
                child[i]=parent['chrom'][i]
        return child
    else:
        return trial
    
    
    

def mutation(parent , target , rand1 , rand2 ,Adj,mutation_rate):
    trial = []
   
    for i in range(0,len(parent['chrom'])):
        
        temp = target['chrom'][i] + mutation_rate*(rand1['chrom'][i] - rand2['chrom'][i])
        temp = int(temp)
#         print(temp)
        if temp < 0 or temp >= Adj.shape[0] or Adj[i,temp]==0:
            temp = target['chrom'][i]
        trial.append(temp)
    return trial
























nodes = []
edges = np.loadtxt('football.txt')
for i in edges:
    for j in i:
        if j not in nodes:
            nodes.append(int(j))
    

# print(nodes)
arr = community_detection(nodes,edges)


print(arr)



#nx.draw_networkx_nodes(graph, arr[0], node_color="tab:blue")

C:\Users\SHUBH\AppData\Local\Temp\ipykernel_22848\458903816.py:73: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  Adj = nx.adjacency_matrix(graph)


[  1   6   4  57  82   0   0  94  11  14  68  64   0  16   9  20   1 111
  21  55  22   1  20  24  32  41  53  77  53  92  28  26  24  24  31  26
   2  48  27   8  24  25   4  40  34  46  86  59  36  41  51  50  63  28
  64  19  38  63  60  66  58  33  47  59  62   5  63 102  10  70  85  77
   7  15  78  56 113  71  90  90 111 108  81  88 104  83  88  70  83   6
  92  75  38 107  97 111  94  96  83  51  84 108  95 110 100 109  96  93
 101  88 103 102  40  82  51]
True
NMI
0.9719482362657703
MODULARITY: 
0.26854397249373957
[[1, 2, 112, 24, 34, 105, 26, 46, 90, 106, 20], [5, 17, 6, 12, 47, 82, 111, 50, 11, 84, 68, 115], [83, 52, 10, 41, 42, 94, 109, 8, 40, 9, 23, 78, 79], [80, 95, 36, 35, 81, 31], [66, 28], [91, 51], [37, 104, 110, 38], [93, 58, 25, 67, 45, 87], [102, 56], [73, 75, 53, 85, 113, 3, 4, 74], [7, 98, 59], [101, 107, 103, 33, 14, 16, 61, 65], [27, 19, 39, 44, 43, 55, 72, 100, 86, 32, 15, 62, 48], [71, 13, 18, 64, 29, 70, 60, 21, 88, 96, 97, 114], [108, 99, 76, 89], [22, 69],